In [1]:
%%html
<style>
table {float:left}
</style>

# <font color="darkgray">Quantum Reservoir Computing</font><br>Mathematical Foundations

<table>
    <tr><td><strong>Aim:</strong></td>
        <td>To formulate a quantum reservoir computing model, so that it could be implemented 
            in <strong>Python</strong> with <strong>PennyLane</strong>.</td></tr>
    <tr><td><strong>Author:</strong></td>
        <td>Jacob L. Cybulski (<a href="https://jacobcybulski.com/" target="_blank">website</a>),
            <em>Enquanted</em></td></tr>
    <tr><td><strong>Release:</strong></td>
        <td>April 2025</td></tr>
    <tr><td><strong>Datasets:</strong></td>
        <td>We will use the the Mackey-Glass time series generator (code included)</td></tr>
    <tr><td><strong>Timeframe:</strong></td>
        <td>01/07/25-26/09/25 (possible extensions)</td></tr>
    <tr><td><strong>References:</strong></td>
        <td>Unfortunately, there is no single simple formulation of a quantum reservoir. 
            Numerous papers have been checked for some good insights, including:
            <ul>
            <li>A. H. Abbas, H. Abdel-Ghani, and I. S. Maksymov, 
                “Classical and Quantum Physical Reservoir Computing for Onboard Artificial Intelligence Systems: A Perspective,” 
                Dynamics, vol. 4, no. 3, pp. 643–670, Aug. 2024, doi: 10.3390/dynamics4030033.</li>
            <li>J. Chen, H. I. Nurdin, and N. Yamamoto, 
                “Temporal Information Processing on Noisy Quantum Computers,” 
                Phys. Rev. Appl., vol. 14, no. 2, p. 024065, Aug. 2020, doi: 10.1103/PhysRevApplied.14.024065.</li>
            <li>K. Fujii and K. Nakajima, 
                “Quantum reservoir computing: a reservoir approach toward quantum machine learning on near-term quantum devices,” 
                Nov. 10, 2020, arXiv: arXiv:2011.04890. doi: 10.48550/arXiv.2011.04890.</li>
            <li>N. Götting, F. Lohof, and C. Gies, 
                “Exploring quantumness in quantum reservoir computing,” 
                Phys. Rev. A, vol. 108, no. 5, p. 052427, Nov. 2023, doi: 10.1103/PhysRevA.108.052427.</li>
            <li>P. Mujal et al., 
                “Opportunities in Quantum Reservoir Computing and Extreme Learning Machines,” 
                Adv. Quantum Technol., vol. 4, no. 8, p. 2100027, Aug. 2021, doi: 10.1002/qute.202100027.</li>
            <li>Q. H. Tran, S. Ghosh, and K. Nakajima, 
                “Quantum-classical hybrid information processing via a single quantum system,” 
                Phys. Rev. Res., vol. 5, no. 4, p. 043127, Nov. 2023, doi: 10.1103/PhysRevResearch.5.043127.</li>
            <li>T. Yasuda et al., 
                “Quantum reservoir computing with repeated measurements on superconducting devices,” 
                Oct. 10, 2023, arXiv: arXiv:2310.06706. doi: 10.48550/arXiv.2310.06706.</li>
        </ul>
        And some others...</td>
    </tr>
    <tr><td><strong>License:</strong></td>
        <td>This project is licensed under the
            <a href="./LICENSE" target="_blank">GNU General Public License V3</a></td></tr>
</table>

## **Quantum reservoir and its dynamics**

Quantum reservoir computing (QRC) is a quantum approach to the creation of Reservoir Computing, which is a concepts derived from recurrent neural networks (RNN). 
It can be applied to all types of data, however, it is especially useful for temporal data. 
Reservoir computing utilises a large sparse neural network of randomly initialised and fixed weights (_**reservoir**_),
responsible for transforming input into a higher-dimensional space.
In high-dimensional space, data can be easily separated  (_**classified**_) by using a simple linear model (_**readout layer**_),
such a ridge regression.
The _**reservoir dynamics**_ can be specified by a set of differential equations (_**update rules**_)
responsible for describing changes to the reservoir state over time.

Typical reservoir computing applications include: time-series forecasting, speech recognition and video analysis, control of robots or autonomous vehicles, as well as, predicting weather patterns and stock markets.

We implement a quantum reservoir as a _**parameterized quantum circuit**_, evolving as a _**classical shadow**_ of the circuit's quantum state (which is explained later).

### 1. **Quantum circuit**
The circuit $ U $ is composed of an input encoding block $ S(u(t)) $ and a parameterized block $ W(\theta) $, applied sequentially:
$$
U(u(t), \theta) = W(\theta) \cdot S(u(t)).
$$
- The circuit acts on $ n $ qubits, initialized in a fixed state (typically $ |0\rangle^{\otimes n} $).

where:
- $U(u(t), \theta)$ is the unitary operator representing the quantum circuit,<br>
  taking input $u(t)$ and fixed parameters $\theta$,
- $S(x)$ is the encoding block for input $x$,
- $W(\theta)$ is a parametrized entangling block,
- $u(t)$ is the input at time $t$,
- $\theta$ are the parameters of the quantum circuit (e.g., rotation angles).

### 2. **Input Encoding Block $ S(u(t)) $**
The input $ u(t) \in \mathbb{R} $ is encoded into the quantum state via unitary operations. While many possible encoding schemes are applicable to time series data, we selected angle encoding as giving the most flexibility:
- For each qubit $ j $, we apply rotation $ e^{-i u(t) G_j} $, where $ G_j $ is a Hermitian operator (e.g., $ X $, $ Y $, or $ Z $).

For example, if we use $RX$ gates for data encoding then the $j$-th qubit encoding would be:
$$
RX(u_j(t)) = e^{-i u_j(t) X},
$$
where $X$ is the Pauli-X operator, which rotates the qubit around the $X$-axis by an angle proportional to the input (note that encoded value may need to be scaled to the appropriate range, e.g. $[0..\pi]$).

In our model, we also add **bias terms** $b_i$ to the input encoding:
$$
RX(u_j(t) + b_j).
$$

### 3. **Parameterized Block $ W(\theta) $**
This block consists of layers of trainable rotations and entangling gates:
- **Single-qubit rotations**: For each qubit $ j $, apply:
$$
  RZ(\theta_{j,3}) RY(\theta_{j,2}) RZ(\theta_{j,1}),
$$
  where $ \theta_{j,k} $ are parameters and $ RY/RZ $ are Pauli-Y/Z rotations.
  
- **Entangling gates**: CNOTs are arranged according to the entanglement graph pattern (in our case, the nearest-neighbor coupling pattern):
$$
  \prod_{(j,k) \in E} \text{CNOT}_{j \to k},
$$
  where $ E $ defines the entanglement graph.

For $ L $ layers, $ W(\theta) $ is a product of $ L $ such blocks:
$$
W(\theta) = \prod_{l=1}^L W_l(\theta_l), \quad \text{where } W_l \text{ is the } l\text{-th layer}.
$$

### 4. **Reservoir State $ r(t) $**
The reservoir state is constructed by measuring single-qubit Pauli expectations:
- For each qubit $ j $, we measure $ \langle X_j \rangle $, $ \langle Y_j \rangle $, and $ \langle Z_j \rangle $:
$$
  \langle X_j \rangle = \text{Tr}(\rho(t) X_j), \quad \langle Y_j \rangle = \text{Tr}(\rho(t) Y_j), \quad \langle Z_j \rangle = \text{Tr}(\rho(t) Z_j),
$$
  where $ \rho(t) = U(u(t), \theta) |0\rangle\langle 0| U^\dagger(u(t), \theta) $ is the circuit's evolved state.

- The reservoir state $ r(t) \in \mathbb{R}^{3n} $ is:
$$
  r(t) = \left[ \langle X_1 \rangle, \langle Y_1 \rangle, \langle Z_1 \rangle, \dots, \langle X_n \rangle, \langle Y_n \rangle, \langle Z_n \rangle \right]^T.
$$

**Notes:** 
- $ r(t) $ is a **classical shadow** of the quantum state, capturing local observables rather than the full state.
- $ r(t) $ dimension is ($ 3n $), which is linear in the number of qubits, making classical model training (such as ridge regression) manageable.
- Should a reservoir of exponential size be required then the **pure state** of the $ n $-qubit system could be used as its state, i.e. $|\psi(t)\rangle = U(u(t), \theta) |0\rangle^{\otimes n}.$

### 5. **Leakage Rate**
To model memory effects (leakage), the QRC maintains the reservoir **effective state**, where the reservoir's measured state is combined with its past states. The **leakage rate** indicates what proportion of the previous state (memory) is lost to accomodate the newly computed state in response to input. For a leakage rate $ \gamma \in (0,1) $:
$$
\tilde{r}(t) = \gamma r(t) + (1 - \gamma) \tilde{r}(t-1),
$$
where:
- $r(t)$ is a the reservoir measured state; and,
- $\tilde{r}(t)$ is the effective state (with leakage) used for readout.

### 6. **Readout Layer**
A classical **ridge regression** maps $ \tilde{r}(t) $ to an output $ y(t) $:
$$
y(t) = w^T \tilde{r}(t) + b,
$$
where weights $ w \in \mathbb{R}^{3n} $ and biases $ b \in \mathbb{R} $ are trained via:
$$
\min_{w,b} \sum_{t} \| y(t) - y_{\text{target}}(t) \|^2 + \lambda \| w \|^2,
$$
with regularization parameter $ \lambda $.

**Note:** see the classical RC model for the explanation of why ridge regression is appropriate for training the hybrid reservoir model.

### **Algorithm summary**
The model’s operation can be summarized as follows:
1. Encode input $ u(t) $ into $ S(u(t)) $.
2. Apply parameterized circuit $ W(\theta) $.
3. Measure Pauli expectations to form state $ r(t) $.
4. Apply leakage to get effective state $ \tilde{r}(t) $.
5. Compute output $ y(t) $ via ridge regression.

### **Notation used**
- **Circuit**: $ U(u(t), \theta) = W(\theta) S(u(t)) $.
- **Circuit state**: $ \rho(t) = U |0\rangle\langle 0| U^\dagger $.
- **Reservoir state**: $ r(t) = \left[ \langle X_1 \rangle, \dots, \langle Z_n \rangle \right]^T $.
- **Effective state**: $ \tilde{r}(t) = \gamma r(t) + (1 - \gamma) \tilde{r}(t-1) $
- **Readout**: $ y(t) = w^T \tilde{r}(t) + b $.